## MAIN FRAME

In [29]:
# Building DBMS Main Frame

from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog

root = Tk()
root.title("Database Management System")
root.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
root.geometry("700x500")
root.configure(bg="white")
root.resizable(width=False, height=False)

# Making title label
title = Label(root, text="Database Management System\nGaruda Indonesia Training Center", 
              font=["Segoe UI Light",25], fg="blue", bg="white").place(relx=0.50625, rely=0.1304, anchor=N)

mode = IntVar()

def check_mode():
    global default
    current_entry = search_box.get()
    if default == False:
        if current_entry == "":
            default = True
        elif current_entry != "":
            default = False

    if mode.get() == 1 and default == True:
        add_btn.config(state="normal", bg="#15ab3a", fg="white")
        search_btn.config(state="normal", bg="#47bdde", fg="white")
        search_box.delete(0, END)
        search_box.insert(0, "Search by contract number or vendor")
    elif mode.get() == 2 and default == True:
        add_btn.config(state="normal", bg="#15ab3a", fg="white")
        search_btn.config(state="normal", bg="#47bdde", fg="white")
        search_box.delete(0, END)
        search_box.insert(0, "Search by MoU number or subject")
        

        
    
# Making Contract mode button
contract_mode = Radiobutton(root, text="Contracts", variable=mode, value=1, bg="white", fg="#08669c", 
                            font={5}, command=check_mode)
contract_mode.place(relx=0.41, rely=0.383, anchor=N)

# Making MoU mode button
mou_mode = Radiobutton(root, text="MoU", variable=mode, value=2, bg="white", fg="#08669c", 
                       font={1}, command=check_mode)
mou_mode.place(relx=0.595, rely=0.383, anchor=N)

# Making title logo
img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((95, 67)))
image_label = Label(root, image=img, bg="white")
image_label.place(relx=0.11, rely=0.008, anchor=N)

# Making search box entry
search_box = Entry(root, width=60, bd=1.5, font={"Segoe UI Light"}, fg="grey", justify="center", relief=GROOVE)
search_box.place(relx=0.5, rely=0.475, anchor=N)
search_box.insert(0, "Welcome to GITC Database Management System")


# Automate the default text to disappear when clicked
def delete_box(event):
    global default
    if default and search_box.get() != "Welcome to GITC Database Management System":
        search_box.delete(0, END)
        default = False

default = True
search_box.bind("<Button-1>", delete_box)


# Open existing records in excel
def open_excel():
    import pandas as pd
    import sqlite3
    import os
    
    conn = sqlite3.connect("database_dum.db")
    df_contracts = pd.read_sql_query("""SELECT * From data_dum""", conn)
    df_mou = pd.read_sql_query("""SELECT * From data_dum_mou""", conn)
    conn.commit()
    conn.close()
    
    writer = pd.ExcelWriter("database_dum_united.xlsx")
    df_contracts.to_excel(writer, sheet_name = "Contracts data", index=False)
    df_mou.to_excel(writer, sheet_name = "MoU data", index=False)
    writer.save()
    os.startfile("database_dum_united.xlsx")

records_btn = Button(root, text="All Records", relief=GROOVE, command=open_excel)
records_btn.place(relx=0.5, rely=0.9, anchor=N)



# Open "Search Result" window
def open_searchwin():
    global searched
    global filepath
    import sqlite3
    import base64
    import codecs
    import webbrowser as wb
    
    
    # Making delete record button functionality
    def delete_rec():
        global scheduler
        from tkinter import messagebox
        
        sure = messagebox.askquestion("Delete Contract", "Are you sure to delete this contract?", parent=search_win)
        if sure == "yes":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            cur.execute("DELETE FROM data_dum WHERE number=?",(number,))
            num_for_mail = number
            conn.commit()
            conn.close()
            
            name_box.delete(0, END) 
            vendor_box.delete(0, END)
            num_box.delete(0, END)
            startdate_box.delete(0, END)
            enddate_box.delete(0, END)
            pdf_box.delete(0, END)
            search_win.destroy()
        else:
            pass
    
    # Making open pdf file functionality
    def SearchPDF():
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        path = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(number,)).fetchone()[0]
        wb.open(path, new=0, autoraise=False)  
        conn.commit()
        conn.close()
    
    
    
    
    ######
    def modify():
        global searched
        global default3
        global pdf_btn_modify
        global savechanges_btn
        global path_modify
        from tkinter import messagebox
        import sqlite3
        
        conn  = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        search_win.title("Edit Data")
        
        searched = str(search_box.get())
        path_validate = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(searched,)).fetchone()
        
        if path_validate == None:
            path_modify = cur.execute("SELECT pdf_file FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
        else:
            path_modify = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(searched,)).fetchone()[0]
            
        conn.commit()
        conn.close
        
        pdf_box.destroy()
        view_pdf_btn.destroy()
        
        file_label_modify = Label(search_win, text=path_modify, bg="white", justify="left", wraplength=310)
        file_label_modify.grid(row=8, column=1, sticky=W)
        
        edit_btn.config(state="disabled")
        
        delete_btn.destroy()
        edit_btn.destroy()

        savechanges_btn = Button(search_win, text="Save Changes", width=11, bg="#72d63c", 
                            fg="white", relief=RAISED, command=save_changes)
        savechanges_btn.place(relx=0.63, rely=0.6563, anchor=N)
        

        default3 = True

        def openfile_modify():
            global default3
            global file_label_modify
            global path_modify
            import base64
            import codecs

            if default3 == True:
                search_win.attributes('-topmost', 'false')                
                path_modify = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
                search_win.attributes('-topmost', 'true')
                file_label_modify = Label(search_win, text=path_modify, bg="white", justify="left", wraplength=310)
                file_label_modify.grid(row=8, column=1, sticky=W)
                default3 = False

            else:
                file_label_modify.destroy()
                search_win.attributes('-topmost', 'false') 
                path_modify = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
                search_win.attributes('-topmost', 'true') 
                file_label_modify = Label(search_win, text=path_modify, bg="white", justify="left", wraplength=310)
                file_label_modify.grid(row=8, column=1, sticky=W)
                default3 = False

        pdf_btn_modify = Button(search_win, text="Choose a file", width=10, command=openfile_modify)
        pdf_btn_modify.grid(row=7, column=1, sticky=W)
        
    def save_changes():
        global path_modify
        from tkinter import messagebox
        import sqlite3
        
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        
        path_validate = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(searched,)).fetchone()
        
        if path_validate == None:
            cur.execute("DELETE FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,))
        else:
            cur.execute("DELETE FROM data_dum WHERE number=?",(searched,))
            
        cur.execute("INSERT INTO data_dum VALUES(:title, :vendor, :number, :start_date, :end_date, :pdf_file)",
                    {"title": name_box.get(),
                    "vendor": vendor_box.get(), 
                    "number": num_box.get(),
                    "start_date": startdate_box.get(),
                    "end_date": enddate_box.get(),
                    "pdf_file": path_modify}
                    )
        
  ##############
        vendor_for_mail = vendor_box.get()
    
        num_for_mail = num_box.get()
        raw_date = cur.execute("SELECT end_date FROM data_dum WHERE number=?", (num_for_mail,))
        str_date = raw_date.fetchall()[0][0]

        from datetime import datetime, timedelta
        import datetime
        
        try:
            str_date_formatted = datetime.datetime.strptime(str_date, '%d-%m-%Y %H:%M:%S')
        except:
            sure = messagebox.askquestion("Invalid input", "Invalid date input! Do you want to submit the data anyway?", parent=add_win)
            if sure == "yes":
                conn.commit()
                conn.close()
                pdf_btn_modify.config(state="disabled")
                savechanges_btn.config(state="disabled")
                search_win.attributes('-topmost', 'false')
                messagebox.showinfo("Action Completed", "All changes has been saved")
                search_win.attributes('-topmost', 'true')
            else:
                pass
        else:
            conn.commit()
            pdf_btn_modify.config(state="disabled")
            savechanges_btn.config(state="disabled")
            search_win.attributes('-topmost', 'false')
            messagebox.showinfo("Action Completed", "All changes has been saved")
            search_win.attributes('-topmost', 'true')
            
            str_date_formatted = datetime.datetime.strptime(str_date, '%d-%m-%Y %H:%M:%S')

            run_at = str_date_formatted + timedelta(minutes=2)
            date_run_at = datetime.datetime.strftime(run_at, '%d-%m-%Y %H:%M:%S')
            file_name_exc = cur.execute("SELECT title FROM data_dum WHERE number=?", (num_for_mail,))
            file_name = file_name_exc.fetchall()[0][0]

            def send_mail(str_date, file_name, vendor_for_mail, num_for_mail):
                import smtplib
                with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
                    smtp.ehlo()
                    smtp.starttls()
                    smtp.ehlo()

                    smtp.login("databasemanagementsystem.gitc@gmail.com", "databasekontrak")

                    subject = "CONTRACT REMINDER | Garuda Indonesia Training Center Database Management System"

                    body = f"Dear Mr/Mrs, \n\nBased on our database of current Garuda Indonesia Training Center contracts, \
we would like to inform you that an existing contract with records as stated below will be ended in \
30 days.\n\nContract Title: {file_name}\nVendor: {vendor_for_mail}\nContract Number: {num_for_mail}\nContract \
Due Date: {str_date}\n\nFor further information, please do not hesitate to contact the Person-in-Charge (PiC).\n\nThank you \
for your attention,\nDatabase Management System"

                    msg = f'Subject: {subject}\n\n{body}'

                    smtp.sendmail("databasemanagementsystem.gitc@gmail.com","arsyi.adlani@gmail.com", msg)
                    
            conn.close()
            from apscheduler.schedulers.background import BackgroundScheduler
            scheduler = BackgroundScheduler()
            scheduler.add_job(send_mail, "date", run_date=run_at, args=[str_date, file_name, vendor_for_mail, num_for_mail])
            scheduler.start()           
            
    
    #############
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    searched = str(search_box.get())
    for row in cur.execute("SELECT number FROM data_dum WHERE number=?", (searched,)):
        validate_num = row
        break
    else:
        validate_num = None
    
    for row in cur.execute("SELECT vendor FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)):
        validate_vendor = row
        break
    else:
        validate_vendor = None
    
    
    # Open the search result toplevel
    if validate_num != None:
        title = cur.execute("SELECT title FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
        vendor = cur.execute("SELECT vendor FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
        number = cur.execute("SELECT number FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
        start_date = cur.execute("SELECT start_date FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
        end_date = cur.execute("SELECT end_date FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
        pdf_file = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?", (searched,)).fetchone()[0]

        search_win = Toplevel()
        search_win.title("Search Data")
        search_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
        search_win.geometry("500x400")
        search_win.configure(bg="white")
        search_win.attributes('-topmost', 'true')
        search_win.resizable(width=False, height=False)

        # Making title logo
        img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
        image_label = Label(search_win, image=img, bg="white")
        image_label.grid(row=0, column=1, sticky=E)

        # Making title lable
        tit_label = Label(search_win, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
        tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

        # Making input labels
        name_lbl = Label(search_win, text="Title", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
        vendor_lbl = Label(search_win, text="Vendor", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
        num_lbl = Label(search_win, text="Number", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
        startdate_lbl = Label(search_win, text="Start Date", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)
        enddate_lbl = Label(search_win, text="Due Date", bg="white").grid(row=6, column=0, sticky=W, padx=22, pady=5)
        pdf_lbl = Label(search_win, text="PDF Document", bg="white").grid(row=7, column=0, sticky=W, padx=22, pady=5)

        # Making existing contract information
        name_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
        name_box.grid(row=2, column=1, sticky=W)
        name_box.insert(0, title)

        vendor_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
        vendor_box.grid(row=3, column=1, sticky=W)
        vendor_box.insert(0, vendor)
        
        num_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
        num_box.grid(row=4, column=1, sticky=W)
        num_box.insert(0, number)

        startdate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
        startdate_box.grid(row=5, column=1, sticky=W)
        startdate_box.insert(0, start_date)

        enddate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
        enddate_box.grid(row=6, column=1, sticky=W)
        enddate_box.insert(0, end_date)
        
        pdf_box = Entry(search_win, width=41, bd=1.5, fg="grey", relief=GROOVE)
        pdf_box.grid(row=7, column=1, sticky=W)
        pdf_box.insert(0, pdf_file)
##
        # Making view existing pdf file button
        view_pdf_btn = Button(search_win, text="View", width=5, bg="#dce0e6", 
                              fg="black", relief=GROOVE, command=SearchPDF)
        view_pdf_btn.place(relx=0.91699, rely=0.6582, anchor=N)

        
        # Making 'Delete Contract' button
        delete_btn = Button(search_win, text="Delete Record", width=13, bg="#f06951", 
                            fg="white", relief=RAISED, command=delete_rec)
        delete_btn.place(relx=0.4485, rely=0.776, anchor=N)
        
        # Making 'Edit Contract' button
        edit_btn = Button(search_win, text="Modify Record", width=26, bg="#15ab3a", 
                          fg="white", relief=RAISED, command=modify)
        edit_btn.place(relx=0.7687, rely=0.776, anchor=N)
   ##############     

    elif validate_vendor != None and searched != "":
        many_validate = []
        for row in cur.execute("SELECT vendor FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)):
            many_validate.append(row)
        if len(many_validate) == 1:
            title = cur.execute("SELECT title FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            vendor = cur.execute("SELECT vendor FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            number = cur.execute("SELECT number FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            start_date = cur.execute("SELECT start_date FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            end_date = cur.execute("SELECT end_date FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            pdf_file = cur.execute("SELECT pdf_file FROM data_dum WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]

            search_win = Toplevel()
            search_win.title("Search Data")
            search_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
            search_win.geometry("500x400")
            search_win.configure(bg="white")
            search_win.attributes('-topmost', 'true')
            search_win.resizable(width=False, height=False)

            # Making title logo
            img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
            image_label = Label(search_win, image=img, bg="white")
            image_label.grid(row=0, column=1, sticky=E)

            # Making title lable
            tit_label = Label(search_win, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
            tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

            # Making input labels
            name_lbl = Label(search_win, text="Title", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
            vendor_lbl = Label(search_win, text="Vendor", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
            num_lbl = Label(search_win, text="Number", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
            startdate_lbl = Label(search_win, text="Start Date", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)
            enddate_lbl = Label(search_win, text="Due Date", bg="white").grid(row=6, column=0, sticky=W, padx=22, pady=5)
            pdf_lbl = Label(search_win, text="PDF Document", bg="white").grid(row=7, column=0, sticky=W, padx=22, pady=5)

            # Making existing contract information
            name_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
            name_box.grid(row=2, column=1, sticky=W)
            name_box.insert(0, title)

            vendor_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
            vendor_box.grid(row=3, column=1, sticky=W)
            vendor_box.insert(0, vendor)

            num_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
            num_box.grid(row=4, column=1, sticky=W)
            num_box.insert(0, number)

            startdate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
            startdate_box.grid(row=5, column=1, sticky=W)
            startdate_box.insert(0, start_date)

            enddate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
            enddate_box.grid(row=6, column=1, sticky=W)
            enddate_box.insert(0, end_date)

            pdf_box = Entry(search_win, width=41, bd=1.5, fg="grey", relief=GROOVE)
            pdf_box.grid(row=7, column=1, sticky=W)
            pdf_box.insert(0, pdf_file)
    ##
            # Making view existing pdf file button
            view_pdf_btn = Button(search_win, text="View", width=5, bg="#dce0e6", 
                                  fg="black", relief=GROOVE, command=SearchPDF)
            view_pdf_btn.place(relx=0.91699, rely=0.6582, anchor=N)


            # Making 'Delete Contract' button
            delete_btn = Button(search_win, text="Delete Record", width=13, bg="#f06951", 
                                fg="white", relief=RAISED, command=delete_rec)
            delete_btn.place(relx=0.4485, rely=0.776, anchor=N)

            # Making 'Edit Contract' button
            edit_btn = Button(search_win, text="Modify Record", width=26, bg="#15ab3a", 
                              fg="white", relief=RAISED, command=modify)
            edit_btn.place(relx=0.7687, rely=0.776, anchor=N)
            
        else:
            import pandas as pd
            import os
            df = pd.read_sql_query("""SELECT * From data_dum WHERE vendor LIKE '%{}%'""".format(searched,), conn)
            df.to_excel("database_dum_exc_quick.xlsx", index=False)
            os.startfile("database_dum_exc_quick.xlsx") 
    
    else:
        from tkinter import messagebox
        messagebox.showerror("Error", "No Data Found!")

    conn.commit()
    conn.close()
    mainloop()

    
    
    
    
    
    
    
def openmou_searchwin():
    global searched
    global filepath
    import sqlite3
    import base64
    import codecs
    import webbrowser as wb
    
    
    # Making delete record button functionality
    def delete_rec():
        from tkinter import messagebox
        
        sure = messagebox.askquestion("Delete Data", "Are you sure to delete this MoU?", parent=search_win_mou)
        if sure == "yes":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            cur.execute("DELETE FROM data_dum_mou WHERE number=?",(number,))
            conn.commit()
            conn.close()
            
            name_box.delete(0, END) 
            num_box.delete(0, END)
            startdate_box.delete(0, END)
            pdf_box.delete(0, END)
            search_win_mou.destroy()
        else:
            pass
    
    # Making open pdf file functionality
    def SearchPDF():
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        path = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(number,)).fetchone()[0]
        wb.open(path, new=0, autoraise=False)  
        conn.commit()
        conn.close()
    
    ######
    def modify():
        global searched
        global default_mou
        global pdf_btn_modify
        global savechanges_btn
        global path_modify_mou
        from tkinter import messagebox
        import sqlite3
        
        conn  = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        search_win_mou.title("Edit Data")
        
        searched = str(search_box.get())
        path_modify_mou = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(searched,)).fetchone()
        
        if path_modify_mou == None:
            path_modify_mou = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)).fetchone()[0]
        else:
            path_modify_mou = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(searched,)).fetchone()[0] 
            
        conn.commit()
        conn.close
        
        pdf_box.destroy()
        view_pdf_btn.destroy()
        
        file_label_modify = Label(search_win_mou, text=path_modify_mou, bg="white", justify="left", wraplength=310)
        file_label_modify.grid(row=6, column=1, sticky=W)
        
        edit_btn.config(state="disabled")
        
        delete_btn.destroy()
        edit_btn.destroy()

        savechanges_btn = Button(search_win_mou, text="Save Changes", width=11, bg="#72d63c", 
                            fg="white", relief=RAISED, command=save_changes)
        savechanges_btn.place(relx=0.63, rely=0.6277, anchor=N)
        
#-------------------------
        default_mou = True

        def openfile_modify():
            global default_mou
            global file_label_modify_mou
            global path_modify_mou

            if default_mou == True:
                search_win_mou.attributes('-topmost', 'false')                
                path_modify_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", 
                                                             filetypes=((".pdf", "*.pdf"),))
                
                search_win_mou.attributes('-topmost', 'true')
                file_label_modify_mou = Label(search_win_mou, text=path_modify_mou, bg="white", 
                                              justify="left", wraplength=310)
                
                file_label_modify_mou.grid(row=6, column=1, sticky=W)
                default_mou = False

            else:
                file_label_mou.destroy()
                search_win_mou.attributes('-topmost', 'false') 
                path_modify_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", 
                                                             filetypes=((".pdf", "*.pdf"),))
                
                search_win_mou.attributes('-topmost', 'true') 
                file_label_modify_mou = Label(search_win_mou, text=path_modify_mou, bg="white", 
                                              justify="left", wraplength=310)
                
                file_label_modify_mou.grid(row=6, column=1, sticky=W)
                default_mou = False

        pdf_btn_modify = Button(search_win_mou, text="Choose a file", width=10, command=openfile_modify)
        pdf_btn_modify.grid(row=5, column=1, sticky=W)
        
    def save_changes():
        global path_modify_mou
        from tkinter import messagebox
        import sqlite3
        
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()

        path_validate = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(searched,)).fetchone()
        if path_validate == None:
            cur.execute("DELETE FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,))
        else:
            cur.execute("DELETE FROM data_dum_mou WHERE number=?",(searched,))
            
        cur.execute("INSERT INTO data_dum_mou VALUES(:subject, :number, :start_date, :pdf_file)",
                    {"subject": name_box.get(),
                    "number": num_box.get(),
                    "start_date": startdate_box.get(),
                    "pdf_file": path_modify_mou}
                    )
        
        import datetime
        try:
            str_date_formatted = datetime.datetime.strptime(startdate_box.get(), '%d-%m-%Y %H:%M:%S')
        except:
            sure = messagebox.askquestion("Invalid input", "Invalid date input! Do you want to save the data anyway?", parent=search_win_mou)
            if sure == "yes":
                conn.commit()
                conn.close()
                pdf_btn_modify.config(state="disabled")
                savechanges_btn.config(state="disabled")
                search_win_mou.attributes('-topmost', 'false')
                messagebox.showinfo("Action Completed", "All changes has been saved")
                search_win_mou.attributes('-topmost', 'true')
            else:
                pass
        else:
            conn.commit()
            conn.close()
            pdf_btn_modify.config(state="disabled")
            savechanges_btn.config(state="disabled")
            search_win_mou.attributes('-topmost', 'false')
            messagebox.showinfo("Action Completed", "All changes has been saved")
            search_win_mou.attributes('-topmost', 'true')
       
            
    #############
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    searched = str(search_box.get())
    for row in cur.execute("SELECT number FROM data_dum_mou WHERE number=?", (searched,)):
        validate_num = row
        break
    else:
        validate_num = None
    
    for row in cur.execute("SELECT subject FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)):
        validate_subj = row
        break
    else:
        validate_subj = None
    
    
    # Open the search result toplevel
    if validate_num != None:
        title = cur.execute("SELECT subject FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        number = cur.execute("SELECT number FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        start_date = cur.execute("SELECT start_date FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        pdf_file = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]

        search_win_mou = Toplevel()
        search_win_mou.title("Search Data")
        search_win_mou.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
        search_win_mou.geometry("500x320")
        search_win_mou.configure(bg="white")
        search_win_mou.attributes('-topmost', 'true')
        search_win_mou.resizable(width=False, height=False)

        # Making title logo
        img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
        image_label = Label(search_win_mou, image=img, bg="white")
        image_label.grid(row=0, column=1, sticky=E)

        # Making title lable
        tit_label = Label(search_win_mou, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
        tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

        # Making input labels
        name_lbl = Label(search_win_mou, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
        num_lbl = Label(search_win_mou, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
        startdate_lbl = Label(search_win_mou, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
        pdf_lbl = Label(search_win_mou, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

        # Making existing contract information
        name_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        name_box.grid(row=2, column=1, sticky=W)
        name_box.insert(0, title)
        
        num_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        num_box.grid(row=3, column=1, sticky=W)
        num_box.insert(0, number)

        startdate_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        startdate_box.grid(row=4, column=1, sticky=W)
        startdate_box.insert(0, start_date)
        
        pdf_box = Entry(search_win_mou, width=41, bd=1.5, fg="grey", relief=GROOVE)
        pdf_box.grid(row=5, column=1, sticky=W)
        pdf_box.insert(0, pdf_file)

        # Making view existing pdf file button
        view_pdf_btn = Button(search_win_mou, text="View", width=5, bg="#dce0e6", 
                              fg="black", relief=GROOVE, command=SearchPDF)
        view_pdf_btn.place(relx=0.91699, rely=0.62763, anchor=N)

        
        # Making 'Delete Contract' button
        delete_btn = Button(search_win_mou, text="Delete Record", width=13, bg="#f06951", 
                            fg="white", relief=RAISED, command=delete_rec)
        delete_btn.place(relx=0.4485, rely=0.78, anchor=N)
        
        # Making 'Edit Contract' button
        edit_btn = Button(search_win_mou, text="Modify Record", width=26, bg="#15ab3a", 
                          fg="white", relief=RAISED, command=modify)
        edit_btn.place(relx=0.7687, rely=0.78, anchor=N)
   

    elif validate_subj != None and searched != "":
        many_validate = []
        for row in cur.execute("SELECT subject FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)):
            many_validate.append(row)
        if len(many_validate) == 1:
            title = cur.execute("SELECT subject FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)).fetchone()[0]
            number = cur.execute("SELECT number FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)).fetchone()[0]
            start_date = cur.execute("SELECT start_date FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)).fetchone()[0]
            pdf_file = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE subject LIKE '%{}%'".format(searched,)).fetchone()[0]

            search_win_mou = Toplevel()
            search_win_mou.title("Search Data")
            search_win_mou.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
            search_win_mou.geometry("500x320")
            search_win_mou.configure(bg="white")
            search_win_mou.attributes('-topmost', 'true')
            search_win_mou.resizable(width=False, height=False)

            # Making title logo
            img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
            image_label = Label(search_win_mou, image=img, bg="white")
            image_label.grid(row=0, column=1, sticky=E)

            # Making title lable
            tit_label = Label(search_win_mou, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
            tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

            # Making input labels
            name_lbl = Label(search_win_mou, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
            num_lbl = Label(search_win_mou, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
            startdate_lbl = Label(search_win_mou, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
            pdf_lbl = Label(search_win_mou, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

            # Making existing contract information
            name_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            name_box.grid(row=2, column=1, sticky=W)
            name_box.insert(0, title)

            num_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            num_box.grid(row=3, column=1, sticky=W)
            num_box.insert(0, number)

            startdate_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            startdate_box.grid(row=4, column=1, sticky=W)
            startdate_box.insert(0, start_date)
            
            pdf_box = Entry(search_win_mou, width=41, bd=1.5, fg="grey", relief=GROOVE)
            pdf_box.grid(row=5, column=1, sticky=W)
            pdf_box.insert(0, pdf_file)

            # Making view existing pdf file button
            view_pdf_btn = Button(search_win_mou, text="View", width=5, bg="#dce0e6", 
                                  fg="black", relief=GROOVE, command=SearchPDF)
            view_pdf_btn.place(relx=0.91699, rely=0.62763, anchor=N)

            # Making 'Delete Contract' button
            delete_btn = Button(search_win_mou, text="Delete Record", width=13, bg="#f06951", 
                                fg="white", relief=RAISED, command=delete_rec)
            delete_btn.place(relx=0.4485, rely=0.78, anchor=N)

            # Making 'Edit Contract' button
            edit_btn = Button(search_win_mou, text="Modify Record", width=26, bg="#15ab3a", 
                              fg="white", relief=RAISED, command=modify)
            edit_btn.place(relx=0.7687, rely=0.78, anchor=N)
            
        else:
            import pandas as pd
            import os
            df = pd.read_sql_query("""SELECT * From data_dum_mou WHERE subject LIKE '%{}%'""".format(searched,), conn)
            df.to_excel("database_dum_exc_quick_mou.xlsx", index=False)
            os.startfile("database_dum_exc_quick_mou.xlsx") 
    
    else:
        from tkinter import messagebox
        messagebox.showerror("Error", "No Data Found!")

    conn.commit()
    conn.close()
    mainloop()    
    
    
    
    

    
    
# Making "Search record" button functionality   
def search_data():
    if mode.get() == 1:
        open_searchwin()
    elif mode.get() == 2:
        openmou_searchwin()
    
    
    
# Making search button
search_btn = Button(root, text="SEARCH", width=60, bg="#e4edf2", fg="black", 
                    font={"Segoe UI Light", "bold"}, relief=RAISED, command=search_data, state="disabled")
search_btn.place(relx=0.50, rely=0.54, anchor=N)


# Making "or" label
or_label = Label(root, text="OR", bg="white", fg="grey", font=["Segoe UI Light",10])
or_label.place(relx=0.5, rely=0.66, anchor=N)


# Open "Add New Contract" Window
def open_addwin():
    global default1
    global default2
    global default3
    global file_label
    global filepath
    
    add_win = Toplevel()
    add_win.title("Add New Record")
    add_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
    add_win.geometry("500x400")
    add_win.configure(bg="white")
    add_win.attributes('-topmost', 'true')
    add_win.resizable(width=False, height=False)

    # Making title logo
    img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
    image_label = Label(add_win, image=img, bg="white")
    image_label.grid(row=0, column=1, sticky=E)

    # Making title lable
    tit_label = Label(add_win, text="Input Form", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
    tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

    # Making input labels
    name_lbl = Label(add_win, text="Title", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
    vendor_lbl = Label(add_win, text="Vendor", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
    num_lbl = Label(add_win, text="Number", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
    startdate_lbl = Label(add_win, text="Start Date", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)
    enddate_lbl = Label(add_win, text="Due Date", bg="white").grid(row=6, column=0, sticky=W, padx=22, pady=5)
    pdf_lbl = Label(add_win, text="PDF Document", bg="white").grid(row=7, column=0, sticky=W, padx=22, pady=5)

    # Making input entry boxes
    name_box = Entry(add_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    name_box.grid(row=2, column=1, sticky=W)
    
    vendor_box = Entry(add_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    vendor_box.grid(row=3, column=1, sticky=W)
    
    num_box = Entry(add_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    num_box.grid(row=4, column=1, sticky=W)

    def del_add_sd(event):
        global default1
        if default1:
            startdate_box.delete(0, END)
            default1 = False    
    default1 = True

    startdate_box = Entry(add_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    startdate_box.grid(row=5, column=1, sticky=W)
    startdate_box.insert(0, "Example: 29-12-2023")
    startdate_box.bind("<Button-1>", del_add_sd)

    def del_add_ed(event):
        global default2
        if default2:
            enddate_box.delete(0, END)
            default2 = False
    default2 = True

    enddate_box = Entry(add_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    enddate_box.grid(row=6, column=1, sticky=W)
    enddate_box.insert(0, "Example: 29-12-2026")
    enddate_box.bind("<Button-1>", del_add_ed)


    # Making File Dialog Window for PDF file and its button
    def openfile():
        global file_label
        global default3
        global filepath
        add_win.attributes("-topmost", "false")
        if default3 == True:
            filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label = Label(add_win, text=filepath, bg="white", wraplength=310, justify="left")
            file_label.grid(row=8, column=1, sticky=W)
            default3 = False
        else:
            file_label.destroy()
            filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label = Label(add_win, text=filepath, bg="white", wraplength=310, justify="left")
            file_label.grid(row=8, column=1, sticky=W)
        add_win.attributes("-topmost", "true")
        if filepath != "":  
            subm_btn.config(state="normal")
        else:
            pass
            # Take 'filepath' as the output to the database
            
    filepath = ""
    default3 = True
    pdf_btn = Button(add_win, text="Choose a file", width=10, command=openfile)
    pdf_btn.grid(row=7, column=1, sticky=W)
    
    
    # Making submit new record button functionality
    def submit():
        import sqlite3
        from tkinter import messagebox
        global filepath
        
        if filepath != "":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            
            num_validate = cur.execute("SELECT number FROM data_dum WHERE number=?", (num_box.get(),)).fetchone()
            if num_validate != None:
                messagebox.showerror("Error", "Contract number had already been taken", parent=add_win)
            else:
                cur.execute("INSERT INTO data_dum VALUES(:title, :vendor, :number, :start_date, :end_date, :pdf_file)",
                           {"title": name_box.get(),
                           "vendor": vendor_box.get(),
                           "number": num_box.get(),
                           "start_date": startdate_box.get(),
                           "end_date": enddate_box.get(),
                           "pdf_file": filepath}
                           )


                ##############
                vendor_for_mail = vendor_box.get()
                num_for_mail = num_box.get()

                raw_date = cur.execute("SELECT end_date FROM data_dum WHERE number=?", (num_for_mail,))
                str_date = raw_date.fetchall()[0][0]

                from datetime import datetime, timedelta
                import datetime
                try:
                    str_date_formatted = datetime.datetime.strptime(str_date, '%d-%m-%Y %H:%M:%S')
                except:
                    sure = messagebox.askquestion("Invalid input", "Invalid date input! Do you want to submit the data anyway?", parent=add_win)
                    if sure == "yes":
                        conn.commit()
                        conn.close()
                        add_win.attributes('-topmost', 'false')
                        messagebox.showinfo("Action Completed", "New contract has been recorded")
                        add_win.attributes('-topmost', 'true')
                        add_win.destroy()
                    else:
                        pass
                else:
                    conn.commit()
                    add_win.attributes('-topmost', 'false')
                    messagebox.showinfo("Action Completed", "New contract has been recorded")
                    add_win.attributes('-topmost', 'true')
                    add_win.destroy()

                    str_date_formatted = datetime.datetime.strptime(str_date, '%d-%m-%Y %H:%M:%S')
                    run_at = str_date_formatted + timedelta(minutes=2)

                    date_run_at = datetime.datetime.strftime(run_at, '%d-%m-%Y %H:%M:%S')

                    file_name_exc = cur.execute("SELECT title FROM data_dum WHERE number=?", (num_for_mail,))
                    file_name = file_name_exc.fetchall()[0][0]

                    def send_mail(str_date, file_name, vendor_for_mail, num_for_mail):
                        import smtplib
                        with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
                            smtp.ehlo()
                            smtp.starttls()
                            smtp.ehlo()

                            smtp.login("databasemanagementsystem.gitc@gmail.com", "databasekontrak")

                            subject = "CONTRACT REMINDER | Garuda Indonesia Training Center Database Management System"

                            body = f"Dear Mr/Mrs, \n\nBased on our database of current Garuda Indonesia Training \
Center contracts, we would like to inform you that an existing contract with records as stated below will be ended in \
30 days.\n\nContract Title: {file_name}\nVendor: {vendor_for_mail}\nContract Number: {num_for_mail}\nContract \
Due Date: {str_date}\n\nFor further information, please do not hesitate to contact the Person-in-Charge (PiC).\n\nThank you \
for your attention,\nDatabase Management System"

                            msg = f'Subject: {subject}\n\n{body}'

                            smtp.sendmail("databasemanagementsystem.gitc@gmail.com","arsyi.adlani@gmail.com", msg)
                            
                    conn.close()

                    from apscheduler.schedulers.background import BackgroundScheduler
                    scheduler = BackgroundScheduler()
                    scheduler.add_job(send_mail, "date", run_date=run_at, args=[str_date, file_name, vendor_for_mail, num_for_mail])
                    scheduler.start()
                               
        #############        
        else:
            pass
         
    # Making 'Submit New Record' button
    subm_btn = Button(add_win, text="Submit", width=9, bg="#15ab3a", 
                      fg="white", relief=RAISED, command=submit, state="disabled")
    subm_btn.place(relx=0.53, rely=0.65625, anchor=N)

    mainloop()

    
    
    
    
    
# Making "Add new MoU" window   
def open_addwin_mou():
    global default1_mou
    global default2_mou
    global default3_mou
    global file_label_mou
    global filepath_mou

    
    addmou_win = Toplevel()
    addmou_win.title("Add New MoU")
    addmou_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
    addmou_win.geometry("460x340")
    addmou_win.configure(bg="white")
    addmou_win.attributes('-topmost', 'true')
    addmou_win.resizable(width=False, height=False)

    # Making title logo
    img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
    image_label = Label(addmou_win, image=img, bg="white")
    image_label.grid(row=0, column=1, sticky=E)

    # Making title lable
    tit_label = Label(addmou_win, text="Input Form", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
    tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

    # Making input labels
    name_lbl = Label(addmou_win, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
    num_lbl = Label(addmou_win, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
    startdate_lbl = Label(addmou_win, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
    pdf_lbl = Label(addmou_win, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

    # Making input entry boxes
    
    
    def del_add_tit(event):
        global default3_mou
        if default3_mou:
            name_box.delete(0, END)
            default3_mou = False    
    default3_mou = True
    
    name_box = Entry(addmou_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    name_box.grid(row=2, column=1, sticky=W)
    name_box.bind("<Button-1>", del_add_tit)
    
########################  

    
    num_box = Entry(addmou_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    num_box.grid(row=3, column=1, sticky=W)

    def del_add_sd(event):
        global default1_mou
        if default1_mou:
            startdate_box.delete(0, END)
            default1_mou = False    
    default1_mou = True

    startdate_box = Entry(addmou_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    startdate_box.grid(row=4, column=1, sticky=W)
    startdate_box.insert(0, "Example: 29-12-2023")
    startdate_box.bind("<Button-1>", del_add_sd)



    # Making File Dialog Window for PDF file and its button
    def openfile():
        global file_label_mou
        global default2_mou
        global filepath_mou
        addmou_win.attributes("-topmost", "false")
        if default2_mou == True:
            filepath_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_mou = Label(addmou_win, text=filepath_mou, bg="white", wraplength=310, justify="left")
            file_label_mou.grid(row=6, column=1, sticky=W)
            default2_mou = False
        else:
            file_label_mou.destroy()
            filepath_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_mou = Label(addmou_win, text=filepath_mou, bg="white", wraplength=310, justify="left")
            file_label_mou.grid(row=6, column=1, sticky=W)
        addmou_win.attributes("-topmost", "true")
        if filepath_mou != "":  
            subm_btn.config(state="normal")
        else:
            pass
            # Take 'filepath' as the output to the database
            
    filepath_mou = ""
    default2_mou = True
    pdf_btn = Button(addmou_win, text="Choose a file", width=10, command=openfile)
    pdf_btn.grid(row=5, column=1, sticky=W)
    
    
    # Making submit new record button functionality
    def submit():
        import sqlite3
        from tkinter import messagebox
        global filepath_mou
        
        if filepath_mou != "":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            
            num_validate = cur.execute("SELECT number FROM data_dum_mou WHERE number=?", (num_box.get(),)).fetchone()
            if num_validate != None:
                messagebox.showerror("Error", "MoU number had already been taken", parent=addmou_win)
                
            else:
                cur.execute("INSERT INTO data_dum_mou VALUES(:subject, :number, :start_date, :pdf_file)",
                           {"subject": name_box.get(),
                           "number": num_box.get(),
                           "start_date": startdate_box.get(),
                           "pdf_file": filepath_mou}
                           )
                import datetime
                try:
                    str_date_formatted = datetime.datetime.strptime(startdate_box.get(), '%d-%m-%Y %H:%M:%S')
                except:
                    sure = messagebox.askquestion("Invalid input", "Invalid date input! Do you want to submit the data anyway?", parent=addmou_win)
                    if sure == "yes":
                        conn.commit()
                        conn.close()
                        addmou_win.attributes('-topmost', 'false')
                        messagebox.showinfo("Action Completed", "New MoU has been recorded")
                        addmou_win.attributes('-topmost', 'true')
                        addmou_win.destroy()
                    else:
                        pass
                else:
                    conn.commit()
                    conn.close()
                    addmou_win.attributes('-topmost', 'false')
                    messagebox.showinfo("Action Completed", "New MoU has been recorded")
                    addmou_win.attributes('-topmost', 'true')
                    addmou_win.destroy()      
        else:
            pass
         
    # Making 'Submit New Record' button
    subm_btn = Button(addmou_win, text="Submit", width=9, bg="#15ab3a", 
                      fg="white", relief=RAISED, command=submit, state="disabled")
    subm_btn.place(relx=0.57, rely=0.5899, anchor=N)

    mainloop()
    
      
# Making "Add new record" button functionality   
def add_new_data():
    if mode.get() == 1:
        open_addwin()
    elif mode.get() == 2:
        open_addwin_mou()
    
    
# Making "Add new record" button
add_btn = Button(root, text="ADD NEW RECORD +", width=21, bg="#ebeef0", fg="white", 
                 font={"Segoe UI Light", "bold"}, relief=RAISED, command=add_new_data, state="disabled")
add_btn.place(relx=0.494, rely=0.749, anchor=N)


# Making copyright label on the bottom right of program
copyright = Label(root, text="Copyright © 2020, Arsyi Adlani", bg="white")
copyright.place(relx=0.86, rely=0.95, anchor=N)

mainloop()

## ADD NEW MOU WINDOW

In [35]:
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog

def open_addwin_mou():
    global default1_mou
    global default2_mou
    global default3_mou
    global file_label_mou
    global filepath_mou

    
    addmou_win = Tk()
    addmou_win.title("Add New MoU")
    addmou_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
    addmou_win.geometry("460x340")
    addmou_win.configure(bg="white")
    addmou_win.attributes('-topmost', 'true')
    addmou_win.resizable(width=False, height=False)

    # Making title logo
    img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
    image_label = Label(addmou_win, image=img, bg="white")
    image_label.grid(row=0, column=1, sticky=E)

    # Making title lable
    tit_label = Label(addmou_win, text="Input Form", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
    tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

    # Making input labels
    name_lbl = Label(addmou_win, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
    num_lbl = Label(addmou_win, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
    startdate_lbl = Label(addmou_win, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
    pdf_lbl = Label(addmou_win, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

    # Making input entry boxes
    
    
    def del_add_tit(event):
        global default3_mou
        if default3_mou:
            name_box.delete(0, END)
            default3_mou = False    
    default3_mou = True
    
    name_box = Entry(addmou_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    name_box.grid(row=2, column=1, sticky=W)
    name_box.bind("<Button-1>", del_add_tit)
    
########################  

    
    num_box = Entry(addmou_win, width=50, bd=1.5, relief=GROOVE)
    num_box.grid(row=3, column=1, sticky=W)

    def del_add_sd(event):
        global default1_mou
        if default1_mou:
            startdate_box.delete(0, END)
            default1_mou = False    
    default1_mou = True

    startdate_box = Entry(addmou_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
    startdate_box.grid(row=4, column=1, sticky=W)
    startdate_box.insert(0, "Example: 29-12-2023")
    startdate_box.bind("<Button-1>", del_add_sd)



    # Making File Dialog Window for PDF file and its button
    def openfile():
        global file_label_mou
        global default2_mou
        global filepath_mou
        addmou_win.attributes("-topmost", "false")
        if default2_mou == True:
            filepath_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_mou = Label(addmou_win, text=filepath_mou, bg="white", wraplength=310, justify="left")
            file_label_mou.grid(row=6, column=1, sticky=W)
            default2_mou = False
        else:
            file_label_mou.destroy()
            filepath_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_mou = Label(addmou_win, text=filepath_mou, bg="white", wraplength=310, justify="left")
            file_label_mou.grid(row=6, column=1, sticky=W)
        addmou_win.attributes("-topmost", "true")
        if filepath_mou != "":  
            subm_btn.config(state="normal")
        else:
            pass
            # Take 'filepath' as the output to the database
            
    filepath_mou = ""
    default2_mou = True
    pdf_btn = Button(addmou_win, text="Choose a file", width=10, command=openfile)
    pdf_btn.grid(row=5, column=1, sticky=W)
    
    
    # Making submit new record button functionality
    def submit():
        import sqlite3
        from tkinter import messagebox
        global filepath_mou
        
        if filepath_mou != "":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            cur.execute("INSERT INTO data_dum_mou VALUES(:subject, :number, :start_date, :pdf_file)",
                       {"subject": name_box.get(),
                       "number": num_box.get(),
                       "start_date": startdate_box.get(),
                       "pdf_file": filepath_mou}
                       )
            
            conn.commit()
            conn.close()

            addmou_win.attributes('-topmost', 'false')
            messagebox.showinfo("Action Completed", "New MoU has been recorded")
            addmou_win.attributes('-topmost', 'true')
            addmou_win.destroy()
        
        else:
            pass
         
    # Making 'Submit New Record' button
    subm_btn = Button(addmou_win, text="Submit", width=9, bg="#15ab3a", 
                      fg="white", relief=RAISED, command=submit, state="disabled")
    subm_btn.place(relx=0.57, rely=0.5899, anchor=N)

    mainloop()
    
open_addwin_mou()

## ADD NEW CONTRACT WINDOW

In [2]:
# Building "Add New Contract" Window
from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog

add_win = Tk()
add_win.title("Add New Contract")
add_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
add_win.geometry("500x400")
add_win.configure(bg="white")
add_win.resizable(width=False, height=False)


# Making title logo
img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((86, 59)))
image_label = Label(add_win, image=img, bg="white")
image_label.image = img
image_label.grid(row=0, column=1, sticky=E)


# Making title lable
tit_label = Label(add_win, text="Input Contract", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)


# Making input labels
name_lbl = Label(add_win, text="Title", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
vendor_lbl = Label(add_win, text="Vendor", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
num_lbl = Label(add_win, text="Number", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
startdate_lbl = Label(add_win, text="Start Date", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)
enddate_lbl = Label(add_win, text="Due Date", bg="white").grid(row=6, column=0, sticky=W, padx=22, pady=5)
pdf_lbl = Label(add_win, text="PDF Document", bg="white").grid(row=7, column=0, sticky=W, padx=22, pady=5)


# Making input entry boxes
name_box = Entry(add_win, width=50, bd=1.5, relief=GROOVE)
name_box.grid(row=2, column=1, sticky=W)

vendor_box = Entry(add_win, width=50, bd=1.5, relief=GROOVE)
vendor_box.grid(row=3, column=1, sticky=W)

num_box = Entry(add_win, width=50, bd=1.5, relief=GROOVE)
num_box.grid(row=4, column=1, sticky=W)

def del_add_sd(event):
    global default1
    if default1:
        startdate_box.delete(0, END)
        default1 = False    
default1 = True

startdate_box = Entry(add_win, width=50, bd=1.5, relief=GROOVE)
startdate_box.grid(row=5, column=1, sticky=W)
startdate_box.insert(0, "Example: 29-12-2023")
startdate_box.bind("<Button-1>", del_add_sd)

def del_add_ed(event):
    global default2
    if default2:
        enddate_box.delete(0, END)
        default2 = False
default2 = True

enddate_box = Entry(add_win, width=50, bd=1.5, relief=GROOVE)
enddate_box.grid(row=6, column=1, sticky=W)
enddate_box.insert(0, "Example: 29-12-2026")
enddate_box.bind("<Button-1>", del_add_ed)


# Making File Dialog Window for PDF file
def openfile():
    global default3
    global file_label
    global filepath
    
    import base64
    import codecs
    
    if default3 == True:
        filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
        file_label = Label(add_win, text=filepath, bg="white", justify="left", wraplength=310)
        file_label.grid(row=8, column=1, sticky=W)
        default3 = False
        
    else:
        file_label.destroy()
        filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
        file_label = Label(add_win, text=filepath, bg="white", justify="left", wraplength=310)
        file_label.grid(row=8, column=1, sticky=W)
        default3 = False

filepath = ""
default3 = True


# Making "Choose pdf file" button
pdf_btn = Button(add_win, text="Choose a file", width=10, command=openfile)
pdf_btn.grid(row=7, column=1, sticky=W)


# Making 'submit new record' function
def submit():
    import sqlite3
    if filepath != "":
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        cur.execute("INSERT INTO data_dum VALUES(:title, :vendor, :number, :start_date, :end_date, :pdf_file)",
                   {"title": name_box.get(),
                   "vendor": vendor_box.get(),
                   "number": num_box.get(),
                   "start_date": startdate_box.get(),
                   "end_date": enddate_box.get(),
                   "pdf_file": filepath}
                   )
        conn.commit()
        conn.close()

        name_box.delete(0, END) 
        vendor_box.delete(0, END)
        num_box.delete(0, END)
        startdate_box.delete(0, END)
        enddate_box.delete(0, END)
        file_label.destroy()
    else:
        pass
    
    
# Making 'Submit' button
subm_btn = Button(add_win, text="SUBMIT", width=9, bg="#15ab3a", fg="white", 
                  relief=RAISED, command=submit)

subm_btn.place(relx=0.825, rely=0.668, anchor=N)

mainloop()

## SEARCH RESULT WINDOW (MOU)

In [ ]:
def openmou_searchwin():
    global searched
    global filepath
    import sqlite3
    import base64
    import codecs
    import webbrowser as wb
    
    
    # Making delete record button functionality
    def delete_rec():
        from tkinter import messagebox
        
        sure = messagebox.askquestion("Delete Data", "Are you sure to delete this MoU?", parent=search_win_mou)
        if sure == "yes":
            conn = sqlite3.connect("database_dum.db")
            cur = conn.cursor()
            cur.execute("DELETE FROM data_dum_mou WHERE number=?",(number,))
            conn.commit()
            conn.close()
            
            name_box.delete(0, END) 
            num_box.delete(0, END)
            startdate_box.delete(0, END)
            pdf_box.delete(0, END)
            search_win_mou.destroy()
        else:
            pass
    
    # Making open pdf file functionality
    def SearchPDF():
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        path = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(number,)).fetchone()[0]
        wb.open(path, new=0, autoraise=False)  
        conn.commit()
        conn.close()
    
    ######
    def modify():
        global searched
        global default_mou
        global pdf_btn_modify
        global savechanges_btn
        global path_modify
        from tkinter import messagebox
        import sqlite3
        
        conn  = sqlite3.connect("database_dum.db")
        cur = conn.cursor()
        search_win_mou.title("Edit Data")
        
        searched = str(search_box.get())
        path_modify = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?",(searched,)).fetchone()[0]
        conn.commit()
        conn.close
        
        pdf_box.destroy()
        view_pdf_btn.destroy()
        
        file_label_modify = Label(search_win_mou, text=path_modify, bg="white", justify="left", wraplength=310)
        file_label_modify.grid(row=8, column=1, sticky=W)
        
        edit_btn.config(state="disabled")
        
        delete_btn.destroy()
        edit_btn.destroy()

        savechanges_btn = Button(search_win_mou, text="Save Changes", width=11, bg="#72d63c", 
                            fg="white", relief=RAISED, command=save_changes)
        savechanges_btn.place(relx=0.63, rely=0.6563, anchor=N)
        

        default_mou = True

        def openfile_modify():
            global default_mou
            global file_label_modify_mou
            global path_modify_mou

            if default_mou == True:
                search_win_mou.attributes('-topmost', 'false')                
                path_modify_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
                search_win_mou.attributes('-topmost', 'true')
                file_label_modify_mou = Label(search_win_mou, text=path_modify, bg="white", justify="left", wraplength=310)
                file_label_modify_mou.grid(row=8, column=1, sticky=W)
                default_mou = False

            else:
                file_label_mou.destroy()
                search_win_mou.attributes('-topmost', 'false') 
                path_modify_mou = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
                search_win_mou.attributes('-topmost', 'true') 
                file_label_modify_mou = Label(search_win_mou, text=path_modify, bg="white", justify="left", wraplength=310)
                file_label_modify_mou.grid(row=8, column=1, sticky=W)
                default_mou = False

        pdf_btn_modify = Button(search_win_mou, text="Choose a file", width=10, command=openfile_modify)
        pdf_btn_modify.grid(row=7, column=1, sticky=W)
        
    def save_changes():
        global path_modify_mou
        from tkinter import messagebox
        import sqlite3
        
        conn = sqlite3.connect("database_dum.db")
        cur = conn.cursor()

        cur.execute("DELETE FROM data_dum_mou WHERE number=?",(searched,))
        cur.execute("INSERT INTO data_dum_mou VALUES(:title, :number, :start_date, :pdf_file)",
                    {"subject": name_box.get(),
                    "number": num_box.get(),
                    "start_date": startdate_box.get(),
                    "pdf_file": path_modify_mou}
                    )
        conn.commit()
        
        
        pdf_btn_modify.config(state="disabled")
        savechanges_btn.config(state="disabled")
        search_win_mou.attributes('-topmost', 'false')
        messagebox.showinfo("Action Completed", "All changes has been saved")
        search_win_mou.attributes('-topmost', 'true')
        
        conn.commit()
        conn.close()
       
            
    
    #############
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    searched = str(search_box.get())
    for row in cur.execute("SELECT number FROM data_dum_mou WHERE number=?", (searched,)):
        validate_num = row
        break
    else:
        validate_num = None
    
    for row in cur.execute("SELECT subject FROM data_dum WHERE subject LIKE '%{}%'".format(searched,)):
        validate_subj = row
        break
    else:
        validate_subj = None
    
    
    # Open the search result toplevel
    if validate_num != None:
        title = cur.execute("SELECT subject FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        number = cur.execute("SELECT number FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        start_date = cur.execute("SELECT start_date FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]
        pdf_file = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE number=?", (searched,)).fetchone()[0]

        search_win_mou = Toplevel()
        search_win_mou.title("Search Data")
        search_win_mou.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
        search_win_mou.geometry("500x400")
        search_win_mou.configure(bg="white")
        search_win_mou.attributes('-topmost', 'true')
        search_win_mou.resizable(width=False, height=False)

        # Making title logo
        img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
        image_label = Label(search_win_mou, image=img, bg="white")
        image_label.grid(row=0, column=1, sticky=E)

        # Making title lable
        tit_label = Label(search_win_mou, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
        tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

        # Making input labels
        name_lbl = Label(search_win_mou, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
        num_lbl = Label(search_win_mou, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
        startdate_lbl = Label(search_win_mou, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
        pdf_lbl = Label(search_win_mou, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

        # Making existing contract information
        name_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        name_box.grid(row=2, column=1, sticky=W)
        name_box.insert(0, title)
        
        num_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        num_box.grid(row=3, column=1, sticky=W)
        num_box.insert(0, number)

        startdate_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
        startdate_box.grid(row=4, column=1, sticky=W)
        startdate_box.insert(0, start_date)
        
        pdf_box = Entry(search_win_mou, width=41, bd=1.5, fg="grey", relief=GROOVE)
        pdf_box.grid(row=5, column=1, sticky=W)
        pdf_box.insert(0, pdf_file)
##
        # Making view existing pdf file button
        view_pdf_btn = Button(search_win_mou, text="View", width=5, bg="#dce0e6", 
                              fg="black", relief=GROOVE, command=SearchPDF)
        view_pdf_btn.place(relx=0.91699, rely=0.6582, anchor=N)

        
        # Making 'Delete Contract' button
        delete_btn = Button(search_win_mou, text="Delete Record", width=13, bg="#f06951", 
                            fg="white", relief=RAISED, command=delete_rec)
        delete_btn.place(relx=0.4485, rely=0.776, anchor=N)
        
        # Making 'Edit Contract' button
        edit_btn = Button(search_win_mou, text="Modify Record", width=26, bg="#15ab3a", 
                          fg="white", relief=RAISED, command=modify)
        edit_btn.place(relx=0.7687, rely=0.776, anchor=N)
   ##############     

    elif validate_subj != None and searched != "":
        many_validate = []
        for row in cur.execute("SELECT subject FROM data_dum_mou WHERE vendor LIKE '%{}%'".format(searched,)):
            many_validate.append(row)
        if len(many_validate) == 1:
            title = cur.execute("SELECT subject FROM data_dum_mou WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            number = cur.execute("SELECT number FROM data_dum_mou WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            start_date = cur.execute("SELECT start_date FROM data_dum_mou WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]
            pdf_file = cur.execute("SELECT pdf_file FROM data_dum_mou WHERE vendor LIKE '%{}%'".format(searched,)).fetchone()[0]

            search_win_mou = Toplevel()
            search_win_mou.title("Search Data")
            search_win_mou.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
            search_win_mou.geometry("500x400")
            search_win_mou.configure(bg="white")
            search_win_mou.attributes('-topmost', 'true')
            search_win_mou.resizable(width=False, height=False)

            # Making title logo
            img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
            image_label = Label(search_win_mou, image=img, bg="white")
            image_label.grid(row=0, column=1, sticky=E)

            # Making title lable
            tit_label = Label(search_win_mou, text="Document Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
            tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)

            # Making input labels
            name_lbl = Label(search_win_mou, text="Subject", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
            num_lbl = Label(search_win_mou, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
            startdate_lbl = Label(search_win_mou, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
            pdf_lbl = Label(search_win_mou, text="PDF Document", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)

            # Making existing contract information
            name_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            name_box.grid(row=2, column=1, sticky=W)
            name_box.insert(0, title)

            num_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            num_box.grid(row=3, column=1, sticky=W)
            num_box.insert(0, number)

            startdate_box = Entry(search_win_mou, width=50, bd=1.5, fg="grey", relief=GROOVE)
            startdate_box.grid(row=4, column=1, sticky=W)
            startdate_box.insert(0, start_date)
            
            pdf_box = Entry(search_win_mou, width=41, bd=1.5, fg="grey", relief=GROOVE)
            pdf_box.grid(row=5, column=1, sticky=W)
            pdf_box.insert(0, pdf_file)
    ##
            # Making view existing pdf file button
            view_pdf_btn = Button(search_win_mou, text="View", width=5, bg="#dce0e6", 
                                  fg="black", relief=GROOVE, command=SearchPDF)
            view_pdf_btn.place(relx=0.91699, rely=0.6582, anchor=N)


            # Making 'Delete Contract' button
            delete_btn = Button(search_win_mou, text="Delete Record", width=13, bg="#f06951", 
                                fg="white", relief=RAISED, command=delete_rec)
            delete_btn.place(relx=0.4485, rely=0.776, anchor=N)

            # Making 'Edit Contract' button
            edit_btn = Button(search_win_mou, text="Modify Record", width=26, bg="#15ab3a", 
                              fg="white", relief=RAISED, command=modify)
            edit_btn.place(relx=0.7687, rely=0.776, anchor=N)
            
        else:
            import pandas as pd
            import os
            df = pd.read_sql_query("""SELECT * From data_dum_mou WHERE subject LIKE '%{}%'""".format(searched,), conn)
            df.to_excel("database_dum_exc_quick_mou.xlsx", index=False)
            os.startfile("database_dum_exc_quick_mou.xlsx") 
    
    else:
        from tkinter import messagebox
        messagebox.showerror("Error", "No Data Found!")

    conn.commit()
    conn.close()
    mainloop()

## SEARCH RESULT WINDOW (CONTRACT)

In [38]:
# Building Search Result Window


from tkinter import *
from PIL import ImageTk, Image
from tkinter import filedialog


search_win = Tk()
search_win.title("Search Contract")
search_win.iconbitmap("c:/Users/Hp/Downloads/garuda.ico")
search_win.geometry("500x400")
search_win.configure(bg="white")
search_win.resizable(width=False, height=False)

# Making title logo
img = ImageTk.PhotoImage(Image.open("c:/Users/Hp/Downloads/logo ga.png").resize((81, 54)))
image_label = Label(search_win, image=img, bg="white")
image_label.grid(row=0, column=1, sticky=E)


# Making title lable
tit_label = Label(search_win, text="Contract Record", font={"Segoe UI Light", 27}, fg="Blue", bg="white")
tit_label.grid(row=1, column=0, padx=22, pady=12, sticky=W)



# Making input labels
name_lbl = Label(search_win, text="Title", bg="white").grid(row=2, column=0, sticky=W, padx=22, pady=5)
num_lbl = Label(search_win, text="Number", bg="white").grid(row=3, column=0, sticky=W, padx=22, pady=5)
startdate_lbl = Label(search_win, text="Start Date", bg="white").grid(row=4, column=0, sticky=W, padx=22, pady=5)
enddate_lbl = Label(search_win, text="Due Date", bg="white").grid(row=5, column=0, sticky=W, padx=22, pady=5)
pdf_lbl = Label(search_win, text="PDF Document", bg="white").grid(row=6, column=0, sticky=W, padx=22, pady=5)


# Making existing contract information
name_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
name_box.grid(row=2, column=1, sticky=W)
name_box.insert(0, "Example: 29-12-2023")


num_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
num_box.grid(row=3, column=1, sticky=W)
num_box.insert(0, "Example: 29-12-2023")


startdate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
startdate_box.grid(row=4, column=1, sticky=W)
startdate_box.insert(0, "Example: 29-12-2023")

enddate_box = Entry(search_win, width=50, bd=1.5, fg="grey", relief=GROOVE)
enddate_box.grid(row=5, column=1, sticky=W)
enddate_box.insert(0, "Example: 29-12-2026")


pdf_box = Entry(search_win, width=42, bd=1.5, fg="grey", relief=GROOVE)
pdf_box.grid(row=6, column=1, sticky=W)
pdf_box.insert(0, "Example: 29-12-2026")

# Making search result functionality
def search_rslt():
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    searched = str(search_box.get())
    title = cur.execute("SELECT title FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
    number = cur.execute("SELECT number FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
    start_date = cur.execute("SELECT start_date FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
    end_date = cur.execute("SELECT end_date FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
    pdf_file = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?", (searched,)).fetchone()[0]
    cur.execute("INSERT INTO data_dum VALUES(:title, :number, :start_date, :end_date, :pdf_file)",
                {"title": title,
                "number": number,
                "start_date": start_date,
                "end_date": end_date,
                "pdf_file": pdf_file}
                )

# Making delete record button functionality
def delete_rec():
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    cur.execute("DELETE FROM data_dum WHERE number=?",(number,))
    conn.commit()
    conn.close()

    name_box.delete(0, END) 
    num_box.delete(0, END)
    startdate_box.delete(0, END)
    enddate_box.delete(0, END)
    file_label.destroy()




# Making 'Delete Contract' button
delete_btn = Button(search_win, text="Delete Record", width=13, bg="#f06951", 
                    fg="white", relief=RAISED)
delete_btn.place(relx=0.425, rely=0.72, anchor=N)


# Making 'Edit Contract' button
edit_btn = Button(search_win, text="Modify Record", width=26, bg="#72d63c", fg="white", relief=RAISED)
edit_btn.place(relx=0.7449, rely=0.72, anchor=N)


# Making 'Save Changes' button
savechanges_btn = Button(search_win, text="Save Changes", width=10, bg="#72d63c", fg="white", relief=RAISED)
savechanges_btn.place(relx=0.853, rely=0.82, anchor=N)


mainloop()

### MODIFY RECORD BUTTON FUNCTIONALITY

In [ ]:
def modify():
    from tkinter import messagebox
    import sqlite3
    
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    searched = str(search_box.get())
    cur.execute("DELETE FROM data_dum WHERE number=?",(searched,))

    pdf_box.destroy()

    default3 = True
    
    def openfile_modify():
        global default3
        global file_label_modify
        global filepath
        import base64
        import codecs

        if default3 == True:
            filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_modify = Label(search_win, text=filepath, bg="white", justify="left", wraplength=310)
            file_label_modify.grid(row=7, column=1, sticky=W)
            default3 = False

        else:
            file_label.destroy()
            filepath = filedialog.askopenfilename(initialdir="c:/", title="Choose File", filetypes=((".pdf", "*.pdf"),))
            file_label_modify = Label(search_win, text=filepath, bg="white", justify="left", wraplength=310)
            file_label_modify.grid(row=7, column=1, sticky=W)
            default3 = False
    
    pdf_btn_modify = Button(add_win, text="Choose a file", width=10, command=openfile_modify)
    pdf_btn_modify.grid(row=6, column=1, sticky=W)
    
    cur.execute("INSERT INTO data_dum VALUES(:title, :number, :start_date, :end_date, :pdf_file)",
                {"title": name_box.get(),
                "number": num_box.get(),
                "start_date": startdate_box.get(),
                "end_date": enddate_box.get(),
                "pdf_file": filepath}
                )
    
    conn.commit()
    conn.close()

    name_box.delete(0, END) 
    num_box.delete(0, END)
    startdate_box.delete(0, END)
    enddate_box.delete(0, END)
    file_label.destroy()


### DATABASE PLAYGROUND (CONTRACT)

In [22]:
import sqlite3
import base64
import codecs
import webbrowser as wb

conn = sqlite3.connect("database_dum.db")
cur = conn.cursor()

cur.execute("""CREATE TABLE data_dum(
            title text,
            vendor text,
            number text,
            start_date text,
            end_date text,
            pdf_file text
           )""")

# title=str(input("title: "))
# number=str(input("number: "))
# start_date=str(input("start: "))
# end_date=str(input("due: "))
# filepath=str(input("filepath: "))

def PDF_to_blob(Filepath):
    datafile = open(Filepath,'rb')
    pdf_blob = datafile.read()
    datafile.close()
    encoded_PDF = codecs.encode(pdf_blob, 'base64')
    decoded_PDF = encoded_PDF.decode('utf-8')
    return decoded_PDF

def SearchPDF(number):
    cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(number,))
    
    while True:
        row = cur.fetchone()
        if row == None:
            break
        data = row[0]
        
    encoded_data = data.encode('utf-8')
    decoded_data = codecs.decode(encoded_data, 'base64')

    datafile = open("newPDFFile.pdf", 'wb')
    datafile.write(decoded_data)
    datafile.close()   
    
    wb.open_new(r'newPDFFile.pdf')  
    
    
    
# pdf_file = PDF_to_blob(filepath)

# cur.execute("""INSERT INTO data_dum VALUES(:title, :number, :start_date, :end_date, :pdf_file)""", 
#             {"title": title, 
#             "number": number,
#             "start_date": start_date,
#             "end_date": end_date,
#             "pdf_file": pdf_file}
#            )


    
conn.commit()
conn.close()

### DATABASE PLAYGROUND (MoU)

In [36]:
import sqlite3
import base64
import codecs
import webbrowser as wb

conn = sqlite3.connect("database_dum.db")
cur = conn.cursor()

cur.execute("""CREATE TABLE data_dum_mou(
            subject text,
            number text,
            start_date text,
            pdf_file text
           )""")

conn.commit()
conn.close()

OperationalError: table data_dum_mou already exists

### VIEWING CONTRACT TABLE WITH PANDAS

In [3]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database_dum.db")
# inp = str(input())
cur = conn.cursor()
# statement = """SELECT * From data_dum WHERE vendor LIKE '%{}%'""".format(inp,)

df = pd.read_sql_query("""SELECT * From data_dum""", conn)

conn.commit()
conn.close()
df

,title,vendor,number,start_date,end_date,pdf_file
0,Perjanjian,PT Abadi Jaya,#A123fh,16-06-2020 21:51:00,16-06-2020 21:51:00,C:/Users/Hp/Pictures/4 Celah.pdf
1,1,abadi ehe,2,-,-,E:/16719147 Arsyi Adlani Introductory Paragrap...
2,Kontrak Asana,AHM,1,16-06-2020 21:00:00,16-06-2020 22:46:00,C:/Users/Hp/Pictures/3 celah.pdf


### VIEWING MoU TABLE WITH PANDAS

In [37]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database_dum.db")
# inp = str(input())
cur = conn.cursor()
# statement = """SELECT * From data_dum WHERE vendor LIKE '%{}%'""".format(inp,)

df = pd.read_sql_query("""SELECT * From data_dum_mou""", conn)

conn.commit()
conn.close()
df

,subject,number,start_date,pdf_file
0,mou 1,1,-,C:/Users/Hp/Pictures/3 celah.pdf


### DELETE TABLE

In [21]:
import sqlite3

conn = sqlite3.connect("database_dum.db")
cur = conn.cursor()

cur.execute("DROP TABLE data_dum")
    
    
conn.commit()
conn.close()

### FUNCTIONS

In [ ]:
def ConvertPDFtoblob(Filepath):
    datafile=open(Filepath,'rb')
    pdfdatab=datafile.read()
    datafile.close()
    b64PDF=codecs.encode(pdfdatab, 'base64')
    Sb64PDF=b64PDF.decode('utf-8')
    return Sb64PDF
#----------------------------------------------------------------
###FUNGSI BLOB TYPE --> PDF###
def ConvertblobtoPDF(Sb64PDF):
    bretdata=Sb64PDF.encode('utf-8')
    global bPDFout 
    bPDFout=codecs.decode(bretdata, 'base64')
    return bPDFout   
def WriteDecodedToFile(bPDFout):
    datafile=open("newPDFFile.pdf",'wb')
    datafile.write(bPDFout)
    datafile.close()   
def OpenPDF():
    wb.open_new(r'newPDFFile.pdf')  
def SearchPDF(Nama):
    global Sb64PDF
    c.execute("SELECT Filepath FROM PDF WHERE Nama=?",(Nama,))
    while True:
        row = c.fetchone()
        if row == None:
            break
        Sb64PDF=row[0]
    ConvertblobtoPDF(Sb64PDF)
    WriteDecodedToFile(bPDFout)
    OpenPDF()

In [32]:
import sqlite3

conn = sqlite3.connect("database_dum.db")
cur = conn.cursor()

number = str(input())
b = ""
for a in cur.execute("SELECT number FROM data_dum WHERE number=?",(number,)):
    var = a[0]
    break
else:
    var=0
conn.commit()
conn.close()
print(var)

2
0


In [56]:
from datetime import datetime, timedelta
import datetime
d=input()
Date=datetime.datetime.strptime(d, '%d-%m-%Y')
run_at0=Date-timedelta(days=30)
Date1=datetime.datetime.strftime(run_at0, '%d-%m-%Y')
Date2=str(Date1)

print(Date)
print(Date1)
print(Date2)

12-05-2023
2023-05-12 00:00:00
12-04-2023
12-04-2023


In [42]:
import sqlite3
import base64
import codecs
import webbrowser as wb

number = str(input())
def SearchPDF(number):
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    
    path = cur.execute("SELECT pdf_file FROM data_dum WHERE number=?",(number,)).fetchone()[0]
    subprocess.Popen([file],shell=True)
    
    conn.commit()
    conn.close()
    
SearchPDF(number)

123


### CONVERT TO EXCEL

In [89]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database_dum.db")
df=pd.read_sql_query("""SELECT * From data_dum""", conn)

conn.commit()
conn.close()
df.to_excel("database_dum_exc.xlsx", index=False)

### OPEN EXCEL FILE

In [11]:
import os

os.startfile("database_dum_exc.xlsx")

In [ ]:
#EMAIL - SCHEDULING#
#------------------------------------------------------------------
def send_mail(number):
    import sqlite3
    conn = sqlite3.connect("database_dum.db")
    cur = conn.cursor()
    raw_date = cur.execute("SELECT end_date FROM data_dum WHERE number=?", (number,))
    str_date = raw_date.fetchall()[0][0]

    from datetime import datetime, timedelta
    import datetime
    str_date_formatted = datetime.datetime.strptime(str_date, '%d-%m-%Y')
    print("str_date_formatted: ", str_date_formatted)
    run_at = Date - timedelta(minutes=2)
    print("run_at", run_at)
    date_run_at = datetime.datetime.strftime(run_at, '%d-%m-%Y %H:%M:%S')
    print(str(date_run_at))
    return str(date_run_at)
 
    file_name = cur.execute("SELECT name FROM data_dum WHERE number=?", (number,))
    
    import smtplib
    with smtplib.SMTP('smtp.gmail.com', 587) as smtp:
        smtp.ehlo()
        smtp.starttls()
        smtp.ehlo()

        smtp.login("databasemanagementsystem.gitc@gmail.com", "databasekontrak")

        subject = "CONTRACT REMINDER | Garuda Indonesia Training Center Database Management System"
        
        body = f"Dear Mr/Mrs \n\nAs stated in the document, the contract entitled: {file_name}, number {number}\ 
        will be ended in 30 days ({raw_date}). For further information, contact the Person-in-Charge (PiC) or by replying to\ 
        this mail \n\nThank you for your attention,\nDatabase Management System"
        
        msg = f'Subject: {subject}\n\n{body}'

        smtp.sendmail("databasemanagementsystem.gitc@gmail.com","arsyi.adlani@gmail.com", msg)
        

    
    #-----------------------------------------------------------------  
def schedule_mail():
    from apscheduler.schedulers.background import BackgroundScheduler
    
    scheduler=BackgroundScheduler()
    scheduler.add_job(send_mail, 'date', run_date=str(date_run_at))
    scheduler.start()
    #----------------------------------------------------------------
    #----------------------------------------------------------------
    conn.commit()
    conn.close()

In [165]:
import pandas as pd
import sqlite3
import os

conn = sqlite3.connect("database_dum.db")

df = pd.read_sql_query("""SELECT * From data_dum WHERE vendor LIKE '%abadi%'""", conn)
df.to_excel("database_dum_exc_quick.xlsx", index=False)
os.startfile("database_dum_exc_quick.xlsx")
conn.commit()
conn.close()


In [123]:
from apscheduler.schedulers.background import BackgroundScheduler
scheduler=BackgroundScheduler()
scheduler.start()
     
def fun():
    def fun1():
        print("hello")
    
    scheduler.add_job(fun1, "date", run_date="2020-06-16 18:20:00", id="a")
    print(scheduler.get_jobs())
    
def fin():
    def fin1():
        print("Heeli")
    scheduler.add_job(fin1, "date", run_date="2020-06-16 18:20:00", id="b")
    scheduler.add_job(fin1, "date", run_date="2020-06-16 18:20:00", id="c")
    scheduler.add_job(fin1, "date", run_date="2020-06-16 18:20:00", id="d")
    scheduler.add_job(fin1, "date", run_date="2020-06-16 18:20:00", id="e")
    print(scheduler.get_jobs())

# fun()
# fin()
print(scheduler.get_jobs())

[]


In [25]:
scheduler.remove_job("2")
print(scheduler.get_jobs())

[]


In [69]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database_dum.db")
df=pd.read_sql_query("""SELECT * From data_dum_mou""", conn)

conn.commit()
conn.close()
df

,subject,number,start_date,pdf_file
0,mou 1,1,-,C:/Users/Hp/Pictures/3 celah.pdf
1,Berita acara 2,2,11-11-1111,C:/Users/Hp/Pictures/4 Celah.pdf


In [175]:
import sqlite3

conn = sqlite3.connect("database_dum.db")
cur = conn.cursor()
inp = str(input())
for row in cur.execute("SELECT number FROM data_dum WHERE number=?",(inp,)):
    validate_num = row
    break
else:
    validate_num = None

vendor_many = []
for row in cur.execute("SELECT vendor FROM data_dum WHERE vendor LIKE '%{}%'".format(inp,)):
    vendor_many.append(row)

a = cur.execute("SELECT title FROM data_dum WHERE vendor LIKE '%{}%'".format(inp,)).fetchone()[0]
print(a)
print(validate_num)
print(len(vendor_many))
conn.commit()
conn.close()

jaya
Perjanjian
None
1


In [143]:
for row in cur.execute("SELECT number FROM data_dum WHERE number='AHM'"):
    validate = row[0]
    break
else:
    validate = None

print(validate)

None


In [79]:
import pandas as pd
import sqlite3

conn = sqlite3.connect("database_dum.db")
df1 = pd.read_sql_query("""SELECT * From data_dum""", conn)
df2 = pd.read_sql_query("""SELECT * From data_dum_mou""", conn)

conn.commit()
conn.close()


writer = pd.ExcelWriter("database_dum_united.xlsx")
df1.to_excel(writer, sheet_name = "Contracts data", index=False)
df2.to_excel(writer, sheet_name = "MoU data", index=False)
writer.save()

import os

# os.startfile("database_dum_united.xlsx")